# 第07章 数据清洗和准备

在数据分析和建模的过程中，相当多的时间要用在数据准备上：加载、清理、转换以及重塑。这些工作会占到分析师时间的80%或更多。有时，存储在文件和数据库中的数据的格式不适合某个特定的任务。许多研究者都选择使用通用编程语言（如Python、Perl、R或Java）或UNIX文本处理工具（如sed或awk）对数据格式进行专门处理。幸运的是，pandas和内置的Python标准库提供了一组高级的、灵活的、快速的工具，可以让你轻松地将数据规整为想要的格式。

如果你发现了一种本书或pandas库中没有的数据操作方式，请在邮件列表或GitHub网站上提出。实际上，pandas的许多设计和实现都是由真实应用的需求所驱动的。

在本章中，我会讨论处理缺失数据、重复数据、字符串操作和其它分析数据转换的工具。下一章，我会关注于用多种方法合并、重塑数据集。

# 7.1 处理缺失数据

在许多数据分析工作中，缺失数据是经常发生的。pandas的目标之一就是尽量轻松地处理缺失数据。例如，pandas对象的所有描述性统计默认都不包括缺失数据。

缺失数据在pandas中呈现的方式有些不完美，但对于大多数用户可以保证功能正常。对于数值数据，pandas使用浮点值NaN（Not a Number）表示缺失数据。我们称其为哨兵值，可以方便的检测出来：

In [1]:
import pandas as pd
import numpy as np

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])

string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

在pandas中，我们采用了R语言中的惯用法，即将缺失值表示为NA，它表示不可用not available。在统计应用中，NA数据可能是不存在的数据或者虽然存在，但是没有观察到（例如，数据采集中发生了问题）。当进行数据清洗以进行分析时，最好直接对缺失数据进行分析，以判断数据采集的问题或缺失数据可能导致的偏差。

Python内置的None值在对象数组中也可以作为NA：

In [4]:
string_data[0] = None

string_data

0         None
1    artichoke
2          NaN
3      avocado
dtype: object

In [5]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

pandas项目中还在不断优化内部细节以更好处理缺失数据，像用户API功能，例如pandas.isnull，去除了许多恼人的细节。表7-1列出了一些关于缺失数据处理的函数。

## 滤除缺失数据

过滤掉缺失数据的办法有很多种。你可以通过pandas.isnull或布尔索引的手工方法，但dropna可能会更实用一些。对于一个Series，dropna返回一个仅含非空数据和索引值的Series：

In [6]:
from numpy import nan as NA

In [8]:
data = pd.Series([1, NA, 3.5, NA, 7])

data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [9]:
data.dropna()

# data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

而对于DataFrame对象，事情就有点复杂了。你可能希望丢弃全NA或含有NA的行或列。dropna默认丢弃任何含有缺失值的行：

In [11]:
data = pd.DataFrame([[1., 6.5, 3.], 
                     [1., NA, NA],
                     [NA, NA, NA], 
                     [NA, 6.5, 3.]]
                   )


cleaned = data.dropna()

In [12]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [13]:
cleaned

,0,1,2
0,1.0,6.5,3.0


传入how='all'将只丢弃全为NA的那些行：

In [14]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


用这种方式丢弃列，只需传入axis=1即可：

In [17]:
data[4] = NA

data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [18]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


另一个滤除DataFrame行的问题涉及时间序列数据。假设你只想留下一部分观测数据，可以用thresh参数实现此目的：

In [25]:
df = pd.DataFrame(np.random.randn(7, 3))

df

,0,1,2
0,2.122416,-0.990207,-0.572224
1,0.183893,-0.821871,-0.858694
2,1.646174,0.546009,0.831709
3,0.287897,0.922408,-0.257314
4,-0.086192,0.096950,-1.805916
5,-0.887391,0.288874,1.505994
6,-1.670261,2.568122,0.315905


In [26]:
df.iloc[:4, 1] = NA

df.iloc[:2, 2] = NA

df

,0,1,2
0,2.122416,NaN,NaN
1,0.183893,NaN,NaN
2,1.646174,NaN,0.831709
3,0.287897,NaN,-0.257314
4,-0.086192,0.096950,-1.805916
5,-0.887391,0.288874,1.505994
6,-1.670261,2.568122,0.315905


In [22]:
df.dropna()

,0,1,2
4,-1.000317,0.089595,-0.104097
5,0.849999,0.930301,0.128376
6,-0.708092,-2.239111,-0.228630


In [32]:
df.dropna(thresh=2)

,0,1,2
2,1.646174,NaN,0.831709
3,0.287897,NaN,-0.257314
4,-0.086192,0.096950,-1.805916
5,-0.887391,0.288874,1.505994
6,-1.670261,2.568122,0.315905


## 充缺失数据

你可能不想滤除缺失数据（有可能会丢弃跟它有关的其他数据），而是希望通过其他方式填补那些“空洞”。对于大多数情况而言，fillna方法是最主要的函数。通过一个常数调用fillna就会将缺失值替换为那个常数值：

In [33]:
df.fillna(0)

,0,1,2
0,2.122416,0.000000,0.000000
1,0.183893,0.000000,0.000000
2,1.646174,0.000000,0.831709
3,0.287897,0.000000,-0.257314
4,-0.086192,0.096950,-1.805916
5,-0.887391,0.288874,1.505994
6,-1.670261,2.568122,0.315905


若是通过一个字典调用fillna，就可以实现对不同的列填充不同的值：

In [34]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,2.122416,0.500000,0.000000
1,0.183893,0.500000,0.000000
2,1.646174,0.500000,0.831709
3,0.287897,0.500000,-0.257314
4,-0.086192,0.096950,-1.805916
5,-0.887391,0.288874,1.505994
6,-1.670261,2.568122,0.315905


fillna默认会返回新对象，但也可以对现有对象进行就地修改：

In [38]:
_ = df.fillna(0, inplace=True)

_

In [39]:
df

,0,1,2
0,2.122416,0.000000,0.000000
1,0.183893,0.000000,0.000000
2,1.646174,0.000000,0.831709
3,0.287897,0.000000,-0.257314
4,-0.086192,0.096950,-1.805916
5,-0.887391,0.288874,1.505994
6,-1.670261,2.568122,0.315905


对reindexing有效的那些插值方法也可用于fillna：

In [48]:
df = pd.DataFrame(np.random.randn(6, 3))

df


,0,1,2
0,-1.704322,0.547064,-0.210639
1,-0.917469,0.884269,0.044337
2,-1.068827,0.560763,-0.901012
3,-0.295520,-2.656343,0.274496
4,0.622802,1.466210,-0.332815
5,-0.399648,-0.290365,-0.924222


In [49]:
df.iloc[2:, 1] = NA

df.iloc[4:, 2] = NA

In [50]:
df

,0,1,2
0,-1.704322,0.547064,-0.210639
1,-0.917469,0.884269,0.044337
2,-1.068827,NaN,-0.901012
3,-0.295520,NaN,0.274496
4,0.622802,NaN,NaN
5,-0.399648,NaN,NaN


In [47]:
df.fillna(method='ffill')

,0,1,2
0,0.639449,-1.947136,-0.900120
1,-2.038453,0.981211,1.865699
2,-0.530615,0.981211,-0.755207
3,1.104038,0.981211,-0.342149
4,-1.282037,0.981211,-0.342149
5,1.066750,0.981211,-0.342149


In [51]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,-1.704322,0.547064,-0.210639
1,-0.917469,0.884269,0.044337
2,-1.068827,0.884269,-0.901012
3,-0.295520,0.884269,0.274496
4,0.622802,NaN,0.274496
5,-0.399648,NaN,0.274496


只要有些创新，你就可以利用fillna实现许多别的功能。比如说，你可以传入Series的平均值或中位数：

In [53]:
data = pd.Series([1., NA, 3.5, NA, 7])

data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [55]:
print(data.mean())

data.fillna(data.mean())

3.8333333333333335


0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64